In [23]:
import pandas as pd
import numpy as np
import pyodbc
import comm as cmm
import time
import codecs
import os
from string import Template



def fmtViewColumns(r):
    split, col, colDscr, dataType, isOrderBy = "\n\t\t", r["Column"], r["ColDscr"], r["Type"], r["orderBy"]
    dataType = "right" if (dataType in ["int","float","decical"]) else "left"
    tmp = Template("{\n\t\tname: '$col',\n\t\tlabel: '$colDscr',\n\t\talign: '$dataType',\n\t\tfield: '$col',\n\t\tsortable: true,\n},\n")
    return tmp.substitute(split = split, col = col, colDscr= colDscr, dataType = dataType) if(isOrderBy =="V") else ""
    
def fmtViewFilterItem(r):
    split, col, colDscr, filterItem = "\n\t\t", r["Column"], r["ColDscr"], r["filterItem"]
    tmp = Template("{$split label: '$colDscr',$split value: '$col',$split placeholder: '請輸入$colDscr',\n},\n")
    return tmp.substitute(split = split ,col = col, colDscr= colDscr) if(filterItem =="V") else ""

def fmtEditFormIniValue(r):
    split, col, type, val = "\t\t\t", r["Column"], r["Type"], ""
    if type in ["int","float","decimal"]:
        val = "0"
    elif type in ["varchar","nvarchar","text"]:
        val = "''"
    elif type in ["datetime2","datetime"]:
        val = "''"
    elif type in ["bit"]:
        val = "true"
    else:
        val="''"

    tmp = Template("$split$col: $val,\n")
    return tmp.substitute(split = split, col = col, val = val) if col not in ["creator","createDT","updator","updateDT"] else ""


In [74]:

pd.options.mode.chained_assignment = None  # default='warn'
class MyTemplate(Template):
    delimiter = "//$"           #自己定義的替換字元符號

df = pd.read_excel("tableSourceDef.xlsx", sheet_name="admin_columns",converters={'maxLen':str }).replace(np.nan, '')
df_sp = pd.read_excel("tableSourceDef.xlsx", sheet_name="SP_config",converters={'maxLen':str }).replace(np.nan, '')
tbs = df_sp["TableName"].unique()


def fmtEditForm_q_Input(r):
    split, qinput, col, colDscr, maxLen = "\t\t\t", r["q-input"], r["Column"], r["ColDscr"], r["maxLen"]
    tmp = f"""<q-input
                class="col"
                type="text"
                v-model.trim="formData.{col}"
                label="{colDscr}"
                lazy-rules
                :rules="[useRequiredInput, (val) => useMaxLength(val, {maxLen})]"
            />
            """
    return "" if (qinput == "") else tmp.format(col = col, colDscr = colDscr, maxLen = maxLen)


def genJSCode(tb, dfm, dfc):
    pt_TableName, pt_tableName, pt_table_name = dfm["TableName"].values[0], dfm["tableName"].values[0], dfm["table_name"].values[0]

    target = dfc[dfc["orderBy"] == "V"]
    target = target[target["Column"]!="updateDT"]

    pt_orderByCols = "".join(target["orderBy2"].tolist())[:-1]
    pt_filterItem = "".join(target["filterItem2"].tolist())[:-1]

    pt_editForm_formData = "".join(dfc["defaultInitValue"].tolist())[1:-1]

    df_q = dfc[dfc["q-input"] != ""]
    iMax = int(df_q["q-input"].max())
    # print(iMax)
    pt_EditForm_q_input =""
    for q in range(iMax):
        k, ss = q+1, ""
        for e in df_q[df_q["q-input"]==k]["qInputStr"].tolist():
            ss+=e

        pt_EditForm_q_input += f"""
        <div class="row q-col-gutter-md">
            {ss}
        </div>
        """.format(ss = ss)
    


    ## -----套表區
    tmpJSFile = codecs.open(f"template/JS/views/admin/index.vue",mode="r",encoding="utf-8", buffering=-1).read()
    t1 = Template(tmpJSFile).substitute(pt_TableName = pt_TableName, pt_tableName = pt_tableName, pt_table_name = pt_table_name, pt_orderByCols = pt_orderByCols, pt_filterItem = pt_filterItem)
    fileName = f"JS/src/views/admin/{pt_table_name}/index.vue"
    os.makedirs(os.path.dirname(fileName), exist_ok=True)
    codecs.open(fileName, mode="w", encoding="utf-8", errors="strict").write(t1)
    # f.write(t1)

    tmp_editForm = codecs.open(f"template/JS/views/admin/edit-form.vue",mode="r",encoding="utf-8", buffering=-1).read()
    t2 = MyTemplate(tmp_editForm).substitute(pt_editForm_formData = pt_editForm_formData, pt_EditForm_q_input = pt_EditForm_q_input)
    fileName = f"JS/src/views/admin/{pt_table_name}/edit-form.vue"
    os.makedirs(os.path.dirname(fileName), exist_ok=True)
    f = codecs.open(fileName, mode="w", encoding="utf-8", errors="strict")
    f.write(t2)
    f.close()


for tb in tbs:

    # print(tb)
    if tb=="PcPhone":
        dfm = df_sp[df_sp["TableName"] == tb]
        dfc = df[df["TableName"] == tb]
        dfc["orderBy2"] = dfc.apply(fmtViewColumns, axis = 1)
        dfc["filterItem2"] = dfc.apply(fmtViewFilterItem, axis = 1)
        dfc["defaultInitValue"] = dfc.apply(fmtEditFormIniValue, axis = 1)
        dfc["qInputStr"] = dfc.apply(fmtEditForm_q_Input, axis = 1)
        # print(dfc["qInputStr"])

        genJSCode(tb, dfm, dfc)
    
